In [0]:
# import
import tensorflow as tf
import numpy as np
import os

In [0]:
# For colab 
from google.colab import files
files.upload()

TypeError: ignored

In [0]:
path_to_file = tf.keras.utils.get_file('Friends_Transcript.txt', 'https://raw.githubusercontent.com/uragirii/Friends-Generator/master/Data/Friends_Transcript.txt')

In [0]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of Character {}'.format(len(text)))

In [0]:
print(text[:250])

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothing to tell! He's just some guy I work with!
Joey:


In [0]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))
print(vocab)

94 unique characters
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}']


In [0]:
# Create a mapping from characters to numbers and vice versa
char2idx = {u:i for i,u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [0]:
print("{} is mapped to {}".format(text[:10], text_as_int[:10]))

THE ONE WH is mapped to [53 41 38  1 48 47 38  1 56 41]


In [0]:
# Maximum sentence we are inputing to the RNN
seq_length =100
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)

48506


In [0]:
# Creating dataset
# from_tensor_slices is like creating a generator for our dataset and is suitable for handling
# large datasets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [0]:
# .take is like iloc in pandas
for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

T
H
E
 
O


In [0]:
# .batch converts into batches of fixed size
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & Da'
"vid Crane\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\nMonica: There's nothin"
"g to tell! He's just some guy I work with!\nJoey: C'mon, you're going out with the guy! There's gotta "
'be something wrong with him!\nChandler: All right Joey, be nice. So does he have a hump? A hump and a '
"hairpiece?\nPhoebe: Wait, does he eat chalk?\n(They all stare, bemused.)\nPhoebe: Just, 'cause, I don't "


In [0]:
def split_input_target(chunk):
    """
    This function generate input and target text from the given text.
    Input text does not contain last part and target doesnot contain first character
    """
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

In [0]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & D'
Target data: 'HE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & Da'


In [0]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 53 ('T')
  expected output: 41 ('H')
Step    1
  input: 41 ('H')
  expected output: 38 ('E')
Step    2
  input: 38 ('E')
  expected output: 1 (' ')
Step    3
  input: 1 (' ')
  expected output: 48 ('O')
Step    4
  input: 48 ('O')
  expected output: 47 ('N')


In [0]:
# Now we need to shuffle the data and pack into batches
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [0]:
#Constants for model
vocab_size = len(vocab)
embedding_dim = 512
# Using half of rnn_units for LSTM
# Speed of training was reduced to half, so i can try 1024 units
rnn_units = 1024


In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.LSTM(rnn_units, return_sequences = True, stateful= True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, BATCH_SIZE)

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [0]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (64, None, 512)           48128     
_________________________________________________________________
lstm_4 (LSTM)                (64, None, 1024)          6295552   
_________________________________________________________________
lstm_5 (LSTM)                (64, None, 1024)          8392704   
_________________________________________________________________
dense_7 (Dense)              (64, None, 94)            96350     
Total params: 14,832,734
Trainable params: 14,832,734
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Here we are chosing the next character randomly based on its probablity
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
#idk what this line does
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices


array([83, 92,  0, 14, 90, 64, 85, 69, 19, 46, 31, 83, 14, 21, 58, 70, 78,
       47, 47, 37, 44, 80, 77, 27, 70, 39, 54, 86,  2, 62, 76, 63, 52,  0,
       40, 43, 28, 26, 53, 43, 11,  2, 71, 89, 55, 66, 28, 58, 86, 61, 56,
       83, 71, 80, 76, 87, 63, 52, 65,  1, 39, 11, 30, 83, 25, 30, 12, 71,
       89,  0, 26, 25, 73, 51,  2, 44, 54, 14, 93, 30, 78,  5, 19, 86,  2,
       17, 18, 49, 14, 78, 14, 22, 86, 24, 20,  0, 22, 91, 13, 70])

In [0]:
#Decoding what this means
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "lling me back?\nJoey: Maybe she never got your message.\nPhoebe: Y'know, if you want, you can call her"

Next Char Predictions: 
 's|\n-z`ue2M>s-4YfnNNDKpm:fFUv!^l_S\nGJ;9TJ*!gyVb;Yv]Wsgplw_Sa F*=s8=+gy\n98iR!KU-}=n$2v!01P-n-5v73\n5{,f'


In [0]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [0]:
model.compile(optimizer='adam', loss=loss)

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [0]:
# model.load_weights('./training_checkpoints/ckpt_18')
import shutil
shutil.rmtree('./training_checkpoints')

FileNotFoundError: ignored

In [129]:
EPOCHS = 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/50
757/757 [==============================] - 94s 124ms/step - loss: 1.7153
Epoch 2/50
757/757 [==============================] - 94s 124ms/step - loss: 1.1954
Epoch 3/50
757/757 [==============================] - 94s 124ms/step - loss: 1.1205
Epoch 4/50
757/757 [==============================] - 94s 124ms/step - loss: 1.0763
Epoch 5/50
757/757 [==============================] - 94s 125ms/step - loss: 1.0429
Epoch 6/50
757/757 [==============================] - 95s 125ms/step - loss: 1.0140
Epoch 7/50
757/757 [==============================] - 94s 124ms/step - loss: 0.9879
Epoch 8/50
757/757 [==============================] - 94s 124ms/step - loss: 0.9632
Epoch 9/50
757/757 [==============================] - 94s 124ms/step - loss: 0.9403
Epoch 10/50
757/757 [==============================] - 94s 124ms/step - loss: 0.9183
Epoch 11/50
757/757 [==============================] - 94s 124ms/step - loss: 0.8976
Epoch 12/50
757/757 [==============================] - 94s 124ms/step - lo

KeyboardInterrupt: ignored

In [131]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_32'

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [133]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (1, None, 512)            48128     
_________________________________________________________________
lstm_6 (LSTM)                (1, None, 1024)           6295552   
_________________________________________________________________
lstm_7 (LSTM)                (1, None, 1024)           8392704   
_________________________________________________________________
dense_8 (Dense)              (1, None, 94)             96350     
Total params: 14,832,734
Trainable params: 14,832,734
Non-trainable params: 0
_________________________________________________________________


In [0]:
def generate_text(model, start_string):
  # Number of characters to generate
  num_generate = 5000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []
  temperature = 0.7

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string=u"Monica: I know"))

In [0]:
model.save('Model_3_LSTM.h5')

In [0]:
os.listdir('./training_checkpoints')

In [146]:
# There is some issue with downloading model.
from google.colab import files
files.download('Model_3_LSTM.h5')

----------------------------------------
Exception happened during processing of request from ('::ffff:127.0.0.1', 55888, 0, 0)
Traceback (most recent call last):
  File "/usr/lib/python3.6/socketserver.py", line 320, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 351, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.6/socketserver.py", line 364, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.6/socketserver.py", line 724, in __init__
    self.handle()
  File "/usr/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/usr/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "/usr/lib/python3.6/http/server.py", line 639, in do_GET
    self.copyfile(f, self.wfile)
  File "/usr/lib/python3.6/http/server.py", line 800, in copyfile
    shutil.copyfil

MessageError: ignored

In [147]:
# Using google drive for saving model
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [149]:
#Saving the look up table for encoding and decoding characters
import json
char2idx

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '#': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 '(': 9,
 ')': 10,
 '*': 11,
 '+': 12,
 ',': 13,
 '-': 14,
 '.': 15,
 '/': 16,
 '0': 17,
 '1': 18,
 '2': 19,
 '3': 20,
 '4': 21,
 '5': 22,
 '6': 23,
 '7': 24,
 '8': 25,
 '9': 26,
 ':': 27,
 ';': 28,
 '<': 29,
 '=': 30,
 '>': 31,
 '?': 32,
 '@': 33,
 'A': 34,
 'B': 35,
 'C': 36,
 'D': 37,
 'E': 38,
 'F': 39,
 'G': 40,
 'H': 41,
 'I': 42,
 'J': 43,
 'K': 44,
 'L': 45,
 'M': 46,
 'N': 47,
 'O': 48,
 'P': 49,
 'Q': 50,
 'R': 51,
 'S': 52,
 'T': 53,
 'U': 54,
 'V': 55,
 'W': 56,
 'X': 57,
 'Y': 58,
 'Z': 59,
 '[': 60,
 ']': 61,
 '^': 62,
 '_': 63,
 '`': 64,
 'a': 65,
 'b': 66,
 'c': 67,
 'd': 68,
 'e': 69,
 'f': 70,
 'g': 71,
 'h': 72,
 'i': 73,
 'j': 74,
 'k': 75,
 'l': 76,
 'm': 77,
 'n': 78,
 'o': 79,
 'p': 80,
 'q': 81,
 'r': 82,
 's': 83,
 't': 84,
 'u': 85,
 'v': 86,
 'w': 87,
 'x': 88,
 'y': 89,
 'z': 90,
 '{': 91,
 '|': 92,
 '}': 93}

In [0]:
import json

with open('encodings.json', 'w') as fp:
    json.dump(char2idx, fp)